**CvC-cleanicDB and Kavasir-seg**

In [54]:
import tensorflow as tf
from tensorflow.keras import layers, models, metrics
from sklearn.model_selection import train_test_split
import pandas as pd 
import matplotlib.pyplot as plt
import numpy as np
import os
from PIL import Image as im
import warnings
warnings.filterwarnings("ignore")
%matplotlib inline


In [19]:
# Data pre-processing
meta_data = pd.read_csv("main_metadata.csv")
meta_data

,Unnamed: 0,image_path,mask_path
0,0,Polyp_dataset/PNG/Original/1.png,Polyp_dataset/PNG/Ground Truth/1.png
1,1,Polyp_dataset/PNG/Original/10.png,Polyp_dataset/PNG/Ground Truth/10.png
2,2,Polyp_dataset/PNG/Original/100.png,Polyp_dataset/PNG/Ground Truth/100.png
3,3,Polyp_dataset/PNG/Original/101.png,Polyp_dataset/PNG/Ground Truth/101.png
4,4,Polyp_dataset/PNG/Original/102.png,Polyp_dataset/PNG/Ground Truth/102.png
...,...,...,...
1608,1608,seg-data/images/cju7dff9529h208503w60lbil.jpg,seg-data/masks/cju7dff9529h208503w60lbil.jpg
1609,1609,seg-data/images/cju2p0eveqtdc0835gpi3p93i.jpg,seg-data/masks/cju2p0eveqtdc0835gpi3p93i.jpg
1610,1610,seg-data/images/cju1hhj6mxfp90835n3wofrap.jpg,seg-data/masks/cju1hhj6mxfp90835n3wofrap.jpg
1611,1611,seg-data/images/cju2yi9tz8vky0801yqip0xyl.jpg,seg-data/masks/cju2yi9tz8vky0801yqip0xyl.jpg


In [47]:
def load_data(meta_data, target_size=(256, 256)):
    image_paths = np.array(meta_data["image_path"])
    mask_paths = np.array(meta_data["mask_path"])

    images, masks = [], []

    for img_path, mask_path in zip(image_paths, mask_paths):
       
        
        # Check if the file exists before reading
        if not os.path.isfile(img_path) or not os.path.isfile(mask_path):
            print(f"❌ Skipping missing file: {img_path} or {mask_path}")
            continue

        # Read the image
        image = cv2.imread(img_path, cv2.IMREAD_UNCHANGED)
        mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)

        if image is None:
            print(f"❌ Failed to load image: {img_path}")
            continue

        if mask is None:
            print(f"❌ Failed to load mask: {mask_path}")
            continue

        # Convert 4-channel PNG (RGBA) to 3-channel (RGB)
        if len(image.shape) == 3 and image.shape[-1] == 4:
            image = cv2.cvtColor(image, cv2.COLOR_BGRA2BGR)

        # Resize only if image and mask are valid
        try:
            image = cv2.resize(image, target_size)
            mask = cv2.resize(mask, target_size)
        except Exception as e:
            print(f"❌ Error resizing: {img_path} or {mask_path} | {str(e)}")
            continue

        # Normalize images and masks
        image = image / 255.0
        mask = mask / 255.0
        mask = np.expand_dims(mask, axis=-1)  # Add channel dimension for compatibility

        images.append(image)
        masks.append(mask)

    images = np.array(images)
    masks = np.array(masks)

    print(f"✅ Loaded {len(images)} images and {len(masks)} masks.")
    return images, masks


        

In [48]:
X,Y = load_data(meta_data)

❌ Skipping missing file: seg-data/images/.ipynb_checkpoints or seg-data/masks/.ipynb_checkpoints
✅ Loaded 1612 images and 1612 masks.


In [50]:
x_train, x_test, y_train, y_test = train_test_split(X,Y,test_size = 0.2, shuffle = True,random_state= 10)
x_train, x_val, y_train, y_val = train_test_split(x_train,y_train, shuffle = True, random_state = 10)

print("x_train size:", len(x_train))
print("x_test size:", len(x_test))
print("x_val size:", len(x_val))

x_train size: 966
x_test size: 323
x_val size: 323


In [51]:
def attention_gate(g, s, num_filters = 128):
    Wg =layers.Conv2D(num_filters, 1, padding="same")(g)
    Wg = layers.BatchNormalization()(Wg)


    up = layers.UpSampling2D(size=(2, 2))(s)
    Ws = layers.Conv2D(num_filters, 1, padding="same")(up)
    Ws = layers.BatchNormalization()(Ws)

    out = layers.Activation("relu")(Wg + Ws)
    out = layers.Conv2D(num_filters, 1, padding="same")(out)
    out = layers.Activation("sigmoid")(out)

    return out

In [58]:
# Use GPU for TensorFlow operations
strategy = tf.distribute.MirroredStrategy()
with strategy.scope():
    # Your TensorFlow code here

    def munet_model(input_shape):
        inputs = layers.Input(input_shape)
#I-B
        conv1 = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(inputs)
        #conv1 = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(conv1)
        pool1 = layers.MaxPooling2D(pool_size=(2, 2))(conv1)

        conv2 = layers.Conv2D(256, (3, 3), activation='relu', padding='same')(pool1)
        pool2 = layers.MaxPooling2D(pool_size=(2, 2))(conv2)

        up1 = layers.UpSampling2D(size=(2, 2))(pool2)
        ag = attention_gate(conv2, pool2)
        up1 = layers.concatenate([ag, up1], axis=-1)
#Encoder
        conv3 = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(up1)
        conv3 = layers.Conv2D(256, (3, 3), activation='relu', padding='same')(conv3)
        pool3 = layers.MaxPooling2D(pool_size=(2, 2))(conv3)


# Bottleneck
        conv4 = layers.Conv2D(512, (3, 3), activation='relu', padding='same')(pool3)
        conv4 = layers.Conv2D(512, (3, 3), activation='relu', padding='same')(conv4)


# Decoder
        up2 = layers.UpSampling2D(size=(2, 2))(conv4)
        ag2 = attention_gate(conv3, conv4)
        up2 = layers.concatenate([ag2, up2], axis=-1)
        conv5 = layers.Conv2D(256, (3, 3), activation='relu', padding='same')(up2)
        conv5 = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(conv5)
        pool4 = layers.MaxPooling2D(pool_size=(2, 2))(conv5)

#O-B
        up3 = layers.UpSampling2D(size=(2, 2))(pool4)
        ag3 = attention_gate(conv5, pool4)
        up3 = layers.concatenate([ag3, up3], axis=-1)
        conv6 = layers.Conv2D(256, (3, 3), activation='relu', padding='same')(up3)

        up4 = layers.UpSampling2D(size=(2, 2))(conv6)
        ag4 = attention_gate(conv1, conv6)
        up4 = layers.concatenate([ag4, up4], axis=-1)
        conv7 = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(up4)

        output = layers.Conv2D(1, (1, 1), activation='sigmoid')(conv7)

        model = models.Model(inputs=inputs, outputs=output)

        return model

# Create an instance of the U-Net model
model = munet_model(input_shape=(256, 256, 3))

# Display the model summary
model.summary()

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0',)


Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_1       │ (None, 256, 256,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_23 (Conv2D)  │ (None, 256, 256,  │      1,792 │ input_layer_1[0]… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_4     │ (None, 128, 128,  │          0 │ conv2d_23[0][0]   │
│ (MaxPooling2D)      │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_24 (Conv2D)  │ (None, 128, 128,  │    147,712 │ max_pooling2d_4[… │
│                     │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_5     │ (None, 64, 64,    │          0 │ conv2d_24[0][0]   │
│ (MaxPooling2D)      │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ up_sampling2d_9     │ (None, 128, 128,  │          0 │ max_pooling2d_5[… │
│ (UpSampling2D)      │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_25 (Conv2D)  │ (None, 128, 128,  │     32,896 │ conv2d_24[0][0]   │
│                     │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_26 (Conv2D)  │ (None, 128, 128,  │     32,896 │ up_sampling2d_9[… │
│                     │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 128, 128,  │        512 │ conv2d_25[0][0]   │
│ (BatchNormalizatio… │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 128, 128,  │        512 │ conv2d_26[0][0]   │
│ (BatchNormalizatio… │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_4 (Add)         │ (None, 128, 128,  │          0 │ batch_normalizat… │
│                     │ 128)              │            │ batch_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_8        │ (None, 128, 128,  │          0 │ add_4[0][0]       │
│ (Activation)        │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_27 (Conv2D)  │ (None, 128, 128,  │     16,512 │ activation_8[0][… │
│                     │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_9        │ (None, 128, 128,  │          0 │ conv2d_27[0][0]   │
│ (Activation)        │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ up_sampling2d_8     │ (None, 128, 128,  │          0 │ max_pooling2d_5[… │
│ (UpSampling2D)      │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate_4       │ (None, 128, 128,  │          0 │ activation_9[0][… │
│ (Concatenate)       │ 384)              │            │ up_sampling2d_8[… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_28 (Conv2D)  │ (None, 128, 128,  │    442,496 │ concatenate_4[0]

 Total params: 7,317,121 (27.91 MB)

 Trainable params: 7,315,073 (27.90 MB)

 Non-trainable params: 2,048 (8.00 KB)

In [ ]:
# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics= ["mse"])

#oom_callback = OOMCallback()

# Train the model
history = model.fit(x_train, y_train,batch_size = 8, epochs= 20 ,shuffle = True, validation_data=(x_val,y_val))

Epoch 1/20


2025-02-28 21:00:21.697957: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.


  3/121 ━━━━━━━━━━━━━━━━━━━━ 17:47 9s/step - loss: 0.8250 - mse: 0.2197 